In [ ]:
import os
import numpy as np
import glob
import PIL.Image as Image
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
if torch.cuda.is_available():
    print("The code will run on GPU.")
else:
    print("The code will run on CPU. Go to Edit->Notebook Settings and choose GPU as the hardware accelerator")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class Hotdog_NotHotdog(torch.utils.data.Dataset):
    def __init__(self, train, transform, data_path='/dtu/datasets1/02514/hotdog_nothotdog'):
        'Initialization'
        self.transform = transform
        data_path = os.path.join(data_path, 'train' if train else 'test')
        image_classes = [os.path.split(d)[1] for d in glob.glob(data_path +'/*') if os.path.isdir(d)]
        image_classes.sort()
        self.name_to_label = {c: id for id, c in enumerate(image_classes)}
        self.image_paths = glob.glob(data_path + '/*/*.jpg')
        
    def __len__(self):
        'Returns the total number of samples'
        return len(self.image_paths)

    def __getitem__(self, idx):
        'Generates one sample of data'
        image_path = self.image_paths[idx]
        
        image = Image.open(image_path)
        c = os.path.split(os.path.split(image_path)[0])[1]
        y = self.name_to_label[c]
        X = self.transform(image)
        return X, y

In [ ]:
size = 128
train_transform = transforms.Compose([transforms.Resize((size, size)), 
                                    transforms.ToTensor()])
test_transform = transforms.Compose([transforms.Resize((size, size)), 
                                    transforms.ToTensor()])

batch_size = 64
trainset = Hotdog_NotHotdog(train=True, transform=train_transform)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=3)
testset = Hotdog_NotHotdog(train=False, transform=test_transform)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=3)

In [ ]:
images, labels = next(iter(train_loader))
plt.figure(figsize=(20,10))

for i in range(21):
    plt.subplot(5,7,i+1)
    plt.imshow(np.swapaxes(np.swapaxes(images[i].numpy(), 0, 2), 0, 1))
    plt.title(['hotdog', 'not hotdog'][labels[i].item()])
    plt.axis('off')

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.convolutional = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.Dropout(),
                nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2),
                nn.ReLU(),
                nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.Dropout(),
                nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1),
                nn.ReLU())

        self.fully_connected = nn.Sequential(
                nn.Linear(64*64*16, 500),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(500, 10),
                nn.Softmax(dim=1))
    
    def forward(self, x):
        x = self.convolutional(x)
        #reshape x so it becomes flat, except for the first dimension (which is the minibatch)
        x = x.view(x.size(0), -1)
        x = self.fully_connected(x)
        return x

In [ ]:
#We define the training as a function so we can easily re-use it.
def train(model, optimizer, num_epochs=10):
    def loss_fun(output, target):
        #return F.nll_loss(torch.log(output), target)
        return F.binary_cross_entropy(output, target)
    out_dict = {'train_acc': [],
              'test_acc': [],
              'train_loss': [],
              'test_loss': []}
    for epoch in tqdm(range(num_epochs), unit='epoch'):
        model.train()
        #For each epoch
        train_correct = 0
        train_loss = []
        for minibatch_no, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):

            data, target = data.to(device), target.to(device)
            
            target = target.to(torch.float32)
            #print(len(target))
            #Zero the gradients computed for each weight
            optimizer.zero_grad()
            #Forward pass your image through the network
            #with torch.no_grad():
            output = model(data)
            
            #Compute the loss
            loss = loss_fun(output, target.reshape(-1,1))
            #Backward pass through the network
            loss.backward()
            #Update the weights
            optimizer.step()

            train_loss.append(loss.item())
            #Compute how many were correctly classified
            predicted = torch.flatten(torch.where(output > 0.5, 1.0, 0.0))
            train_correct += (target==predicted).sum().cpu().item()
            #print(len(output))
            #print(len(predicted))
            #print("------------------")
            
        #print(train_correct)
        #Comput the test accuracy
        test_loss = []
        test_correct = 0
        model.eval()
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            target = target.to(torch.float32)
            with torch.no_grad():
                output = model(data)
            test_loss.append(loss_fun(output, target.reshape(-1,1)).cpu().item())
            predicted = torch.flatten(torch.where(output > 0.5, 1.0, 0.0))
            test_correct += (target==predicted).sum().cpu().item()
            
        out_dict['train_acc'].append(train_correct/len(train_set))
        out_dict['test_acc'].append(test_correct/len(testset))
        out_dict['train_loss'].append(np.mean(train_loss))
        out_dict['test_loss'].append(np.mean(test_loss))
        print(f"Loss train: {np.mean(train_loss):.3f}\t test: {np.mean(test_loss):.3f}\t",
              f"Accuracy train: {out_dict['train_acc'][-1]*100:.1f}%\t test: {out_dict['test_acc'][-1]*100:.1f}%")
    return out_dict

In [ ]:
model = Network()
model.to(device)
#Initialize the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.90)
optimizer_adam = torch.optim.Adam(model.parameters())

In [ ]:
out_dict = train(model, optimizer_adam,num_epochs=10)

In [ ]:
layers=[x.data for x in model.parameters()]
layers[0].cpu().detach().numpy().shape

In [ ]:
fig, ax = plt.subplots(2,4)
ax[0,0].imshow(layers[0].cpu().detach().numpy()[0][0],interpolation='none')
ax[0,1].imshow(layers[0].cpu().detach().numpy()[1][0],interpolation='none')
ax[0,2].imshow(layers[0].cpu().detach().numpy()[2][0],interpolation='none')
ax[0,3].imshow(layers[0].cpu().detach().numpy()[3][0],interpolation='none')
ax[1,0].imshow(layers[0].cpu().detach().numpy()[4][0],interpolation='none')
ax[1,1].imshow(layers[0].cpu().detach().numpy()[5][0],interpolation='none')
ax[1,2].imshow(layers[0].cpu().detach().numpy()[6][0],interpolation='none')
ax[1,3].imshow(layers[0].cpu().detach().numpy()[7][0],interpolation='none')

In [ ]:
plt.plot(np.arange(10),np.array(out_dict["test_loss"]),'-')
plt.plot(np.arange(10),np.array(out_dict["train_loss"]),'-')
plt.legend(('Test error','Train eror'))
plt.xlabel('Epoch number')
plt.ylabel('Accuracy')

In [ ]:
plt.plot(np.arange(10),np.array(out_dict["test_acc"]),'-')
plt.plot(np.arange(10),np.array(out_dict["train_acc"]),'-')
plt.legend(('Test error','Train eror'))
plt.xlabel('Epoch number')
plt.ylabel('Accuracy')

In [ ]:
test_correct = 0
targets = np.array([])
predictions = np.array([])
data_images = []

In [ ]:
def loss_fun(output, target):
    #return F.nll_loss(torch.log(output), target)
    return F.binary_cross_entropy(output, target)
test_loss = []
test_correct = 0
model.eval()
i = 0
for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    target = target.to(torch.float32)
    with torch.no_grad():
        output = model(data)
    test_loss.append(loss_fun(output, target.reshape(-1,1)).cpu().item())
    predicted = torch.flatten(torch.where(output > 0.5, 1.0, 0.0))
    test_correct += (target==predicted).sum().cpu().item()
    i += 1
    targets = np.concatenate((targets,target.cpu().detach().numpy()))
    predictions = np.concatenate((predictions,predicted.cpu().detach().numpy()))
    #if i != 29:
    data_images.append(data[:].cpu().detach().numpy())

In [ ]:
images, labels = next(iter(test_loader))

In [ ]:
plt.imshow(np.swapaxes(np.swapaxes(images[i].numpy(), 0, 2), 0, 1))

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
hist, xbins, ybins, im = ax.hist2d(targets, predictions,bins=[2,2])
for i in range(len(ybins)-1):
    for j in range(len(xbins)-1):
        ax.text(xbins[j]+0.25,ybins[i]+0.25, hist.T[i,j], 
                color="w", ha="center", va="center", fontweight="bold")
ax.set_ylabel('predictions')
ax.set_xlabel('targets')

In [ ]:
import torch.optim as optim

In [ ]:
from torch.optim import lr_scheduler

In [ ]:
import torch.backends.cudnn as cudnn

In [ ]:
from torchvision import datasets, models, transforms

In [ ]:
model_ft = models.resnet18(pretrained=True)

In [ ]:
num_ftrs = model_ft.fc.in_features

In [ ]:
model_ft.fc = nn.Linear(num_ftrs, 1)

In [ ]:
model_ft = model_ft.to(device)

### Saliency map

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
#load pretrained resnet model
model = torchvision.models.resnet50(pretrained=True)
#print(model)
model.to(device)

#define transforms to preprocess input image into format expected by model
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
#inverse transform to get normalize image back to original form for visualization
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255],
    std=[1/0.229, 1/0.224, 1/0.255]
)

#transforms to resize image to the size expected by pretrained model,
#convert PIL image to tensor, and
#normalize the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    normalize,          
])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def saliency(img, model):
    #we don't need gradients w.r.t. weights for a trained model
    for param in model.parameters():
        param.requires_grad = False
    
    #set model in eval mode
    model.eval()
    #transoform input PIL image to torch.Tensor and normalize
    input = transform(img)
    input.unsqueeze_(0)

    #we want to calculate gradient of higest score w.r.t. input
    #so set requires_grad to True for input 
    input.requires_grad = True
    #print(input)
    #forward pass to calculate predictions
    preds = model(input)
    #print(preds)
    score, indices = torch.max(preds, 1)
    #backward pass to get gradients of score predicted class w.r.t. input image
    score.backward()
    #get max along channel axis
    slc, _ = torch.max(torch.abs(input.grad[0]), dim=0)
    #print(input.grad[0])
    #normalize to [0..1]
    slc = (slc - slc.min())/(slc.max()-slc.min())
    #apply inverse transform on image
    with torch.no_grad():
        input_img = inv_normalize(input[0])
    #plot image and its saleincy map
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(np.transpose(input_img.cpu().detach().numpy(), (1, 2, 0)))
    plt.xticks([])
    plt.yticks([])
    plt.subplot(1, 2, 2)
    plt.imshow(slc.cpu().detach().numpy(), cmap=plt.cm.hot)
    plt.xticks([])
    plt.yticks([])
    plt.show()
    return slc

In [ ]:
images, labels = next(iter(test_loader))

In [ ]:
img = images[36].to(device)
slc = saliency(img, model)

In [ ]:
slc